# 21

In [13]:
num_keypad = (('7','8','9'),
              ('4','5','6'),
              ('1','2','3'),
              ('X','0','A'))

num_position = (3,2) # row, col 

dir_keypad = (('X', '^', 'A'),
              ('<', 'v', '>'))

dir_position = (0,2) # row, col

In [14]:
from functools import cache

@cache
def get_sequences(code, keypad, position):
    # Initialize the current position to the starting position
    current = position
   
    # Known paths: 
    result = ['']

    # Go character by character in the code
    for char in code: 
        # Find the position of the character on the keypad
        char_position = None

        for row in range(len(keypad)):
            for col in range(len(keypad[0])):
                if keypad[row][col] == char:
                    char_position = (row, col)

        # Find all paths from the current position to the character 
        paths = [('', current)] # (path, position)
        path_stubs = []

        while len(paths) > 0: 
            new_paths = []
            for path, position in paths:
                if position[0] < char_position[0]:
                    if keypad[position[0]+1][position[1]] != 'X':
                        new_paths.append((path+'v', (position[0]+1, position[1])))
                elif position[0] > char_position[0]:
                    if keypad[position[0]-1][position[1]] != 'X':
                        new_paths.append((path+'^', (position[0]-1, position[1])))

                if position[1] < char_position[1]:
                    if keypad[position[0]][position[1]+1] != 'X':
                        new_paths.append((path+'>', (position[0], position[1]+1)))
                elif position[1] > char_position[1]:
                    if keypad[position[0]][position[1]-1] != 'X':
                        new_paths.append((path+'<', (position[0], position[1]-1)))

                if position == char_position:
                    path = path + 'A'
                    path_stubs.append(path)
            paths = new_paths

        # print(f"char: {char}", path_stubs)
        # Update the current position 
        current = char_position

        # Add the shortest path to the result
        updated_result = []
        for path in result: 
            for stub in path_stubs:
                
                updated_result.append(path + stub)

        result = updated_result

    return result         

In [20]:
@cache
def first_sequence(code):
    return get_sequences(code, num_keypad, num_position)

@cache
def second_sequence(code):
    return get_sequences(code, dir_keypad, dir_position)


### Solution

In [22]:
codes = []
with open("input.txt") as f: 
    for line in f: 
        codes.append(line.strip())
codes

['593A', '508A', '386A', '459A', '246A']

In [23]:
result = 0 

for code in codes: 
    second_sequences = []

    for sequence in first_sequence(code):
        #print(sequence)
        second_sequences.extend(second_sequence(sequence))

    second_sequences.sort(key=lambda x: len(x))
    
    third_sequences = []
    for sequence in second_sequences:
        #print(sequence)
        third_sequences.extend(second_sequence(sequence))

    third_sequences.sort(key=lambda x: len(x))

    # compute the complexity 
    complexity = len(third_sequences[0]) * int(code[:-1])

    result += complexity

result

157892

## Part 2

In [101]:
dir_keypad
num_keypad

[['X', '^', 'A'], ['<', 'v', '>']]

In [6]:
get_sequences('029A', num_keypad, num_position)

NameError: name 'get_sequences' is not defined

In [102]:
def find_dir_sequences(current, target, keypad = dir_keypad):
    # Find all paths from the current position to the character 
    path_stubs = []

    paths = [('', current)] 
    while len(paths) > 0: 
        new_paths = []
        for path, init_position in paths:
            if init_position[0] < target[0]:
                if keypad[init_position[0]+1][init_position[1]] != 'X':
                    new_paths.append((path+'v', (init_position[0]+1, init_position[1])))
            elif init_position[0] > target[0]:
                if keypad[init_position[0]-1][init_position[1]] != 'X':
                    new_paths.append((path+'^', (init_position[0]-1, init_position[1])))

            if init_position[1] < target[1]:
                if keypad[init_position[0]][init_position[1]+1] != 'X':
                    new_paths.append((path+'>', (init_position[0], init_position[1]+1)))
            elif init_position[1] > target[1]:
                if keypad[init_position[0]][init_position[1]-1] != 'X':
                    new_paths.append((path+'<', (init_position[0], init_position[1]-1)))

            if init_position == target:
                path = path + 'A'
                path_stubs.append(path)
        paths = new_paths

    return path_stubs

def precompute_dir_sequences(keypad = dir_keypad): 
    sequences = {}
    for init_row in range(len(keypad)):
        for init_col in range(len(keypad[0])):
            for target_row in range(len(keypad)):
                for target_col in range(len(keypad[0])):
                    sequences[(init_row, init_col), (target_row, target_col)] = find_dir_sequences((init_row, init_col), (target_row, target_col), keypad)
    return sequences

In [103]:
dir_keypad_sequences = precompute_dir_sequences()
num_keypad_sequences = precompute_dir_sequences(num_keypad)

In [109]:
from functools import cache 

def get_sequences_memo(code, keypad, init_position):
    memo = num_keypad_sequences if keypad == num_keypad else dir_keypad_sequences
    # Initialize the current position to the starting position
    current = init_position
   
    # Known paths: 
    result = ''

    # Go character by character in the code
    for char in code: 
        # Find the position of the character on the keypad
        char_position = None

        for row in range(len(keypad)):
            for col in range(len(keypad[0])):
                if keypad[row][col] == char:
                    char_position = (row, col)

       
        path_stubs = memo[(current, char_position)]
        current = char_position

        result = result + path_stubs[0]

    return result

In [110]:
get_sequences_memo('029A', num_keypad, num_position)

'<A^A^^>AvvvA'

In [15]:
# Memoize on top of this? Wth
from tqdm import tqdm

result = 0

for code in codes:
    first_sequences = get_sequences_memo(code, num_keypad, num_position)
    current_sequences = first_sequences
    
    print("code: ", code)
    for _ in tqdm(range(15)):

        new_current_sequences = []
        for sequence in current_sequences:
            new_current_sequences.extend(get_sequences_memo(sequence, dir_keypad, dir_position))
       
        shortest_path_length = 10*len(sequence)

        for sequence in new_current_sequences:
            if len(sequence) < shortest_path_length:
                shortest_path_length = len(sequence)

        current_sequences = filter(lambda x: len(x) == shortest_path_length, new_current_sequences)

    result += shortest_path_length * int(code[:-1])

result

code:  593A


100%|██████████| 15/15 [00:08<00:00,  1.79it/s]


code:  508A


100%|██████████| 15/15 [00:12<00:00,  1.20it/s]


code:  386A


...

code:  459A


100%|██████████| 15/15 [00:10<00:00,  1.39it/s]


code:  246A


100%|██████████| 15/15 [00:08<00:00,  1.67it/s]


2192

In [67]:
filtered_dir = dict() 

for (start, end), seq in dir_keypad_sequences.items():
    if start == (0,0) or end == (0,0):
        continue

    candidates = []
    shortest = -1
    for s in seq:

        if shortest == -1 or len(second_sequence(s)[0]) < shortest:
            shortest = len(second_sequence(s)[0])
            candidates = [s]
        elif len(second_sequence(s)[0]) == shortest:
            candidates.append(s)
    filtered_dir[(start, end)] = candidates
    

double_filter = dict()
for (start, end), seq in filtered_dir.items():
    if start == (0,0) or end == (0,0):
        continue

    candidates = []
    shortest = -1
    for s in seq:

        if shortest == -1 or len(second_sequence(s)[0]) < shortest:
            shortest = len(second_sequence(s)[0])
            candidates = [s]
        elif len(second_sequence(s)[0]) == shortest:
            candidates.append(s)
    double_filter[(start, end)] = candidates
    
double_filter 

{((0, 1), (0, 1)): ['A'],
 ((0, 1), (0, 2)): ['>A'],
 ((0, 1), (1, 0)): ['v<A'],
 ((0, 1), (1, 1)): ['vA'],
 ((0, 1), (1, 2)): ['v>A', '>vA'],
 ((0, 2), (0, 1)): ['<A'],
 ((0, 2), (0, 2)): ['A'],
 ((0, 2), (1, 0)): ['v<<A'],
 ((0, 2), (1, 1)): ['v<A', '<vA'],
 ((0, 2), (1, 2)): ['vA'],
 ((1, 0), (0, 1)): ['>^A'],
 ((1, 0), (0, 2)): ['>>^A'],
 ((1, 0), (1, 0)): ['A'],
 ((1, 0), (1, 1)): ['>A'],
 ((1, 0), (1, 2)): ['>>A'],
 ((1, 1), (0, 1)): ['^A'],
 ((1, 1), (0, 2)): ['^>A', '>^A'],
 ((1, 1), (1, 0)): ['<A'],
 ((1, 1), (1, 1)): ['A'],
 ((1, 1), (1, 2)): ['>A'],
 ((1, 2), (0, 1)): ['^<A', '<^A'],
 ((1, 2), (0, 2)): ['^A'],
 ((1, 2), (1, 0)): ['<<A'],
 ((1, 2), (1, 1)): ['<A'],
 ((1, 2), (1, 2)): ['A']}

In [88]:
seq = second_sequence('>^A')
for s in seq:
    print('--')
    seq2 = second_sequence(s)
    print(s, len(second_sequence(s)[0]))
    print('---')    
    for s2 in seq2:
        print(s2, len(second_sequence(s2)[0]))



--
vA^<A>A 19
---
v<A^>A<Av<A>^>AvA^A 53
v<A^>A<Av<A>>^AvA^A 51
v<A>^A<Av<A>^>AvA^A 53
v<A>^A<Av<A>>^AvA^A 51
<vA^>A<Av<A>^>AvA^A 53
<vA^>A<Av<A>>^AvA^A 51
<vA>^A<Av<A>^>AvA^A 53
<vA>^A<Av<A>>^AvA^A 51
--
vA<^A>A 19
---
v<A^>Av<<A>^A>AvA^A 47
v<A^>A<v<A>^A>AvA^A 49
v<A>^Av<<A>^A>AvA^A 47
v<A>^A<v<A>^A>AvA^A 49
<vA^>Av<<A>^A>AvA^A 47
<vA^>A<v<A>^A>AvA^A 49
<vA>^Av<<A>^A>AvA^A 47
<vA>^A<v<A>^A>AvA^A 49


In [71]:
filtered_dir

{((0, 1), (0, 1)): ['A'],
 ((0, 1), (0, 2)): ['>A'],
 ((0, 1), (1, 0)): ['v<A'],
 ((0, 1), (1, 1)): ['vA'],
 ((0, 1), (1, 2)): ['v>A', '>vA'],
 ((0, 2), (0, 1)): ['<A'],
 ((0, 2), (0, 2)): ['A'],
 ((0, 2), (1, 0)): ['v<<A'],
 ((0, 2), (1, 1)): ['v<A', '<vA'],
 ((0, 2), (1, 2)): ['vA'],
 ((1, 0), (0, 1)): ['>^A'],
 ((1, 0), (0, 2)): ['>>^A'],
 ((1, 0), (1, 0)): ['A'],
 ((1, 0), (1, 1)): ['>A'],
 ((1, 0), (1, 2)): ['>>A'],
 ((1, 1), (0, 1)): ['^A'],
 ((1, 1), (0, 2)): ['^>A', '>^A'],
 ((1, 1), (1, 0)): ['<A'],
 ((1, 1), (1, 1)): ['A'],
 ((1, 1), (1, 2)): ['>A'],
 ((1, 2), (0, 1)): ['^<A', '<^A'],
 ((1, 2), (0, 2)): ['^A'],
 ((1, 2), (1, 0)): ['<<A'],
 ((1, 2), (1, 1)): ['<A'],
 ((1, 2), (1, 2)): ['A']}

In [58]:
for (start, end), seq in dir_keypad_sequences.items():
    if start == (0,0) or end == (0,0):
        continue

    candidates = []
    shortest = 100000
    for s in seq:
       
        print(s, len(second_sequence(s)[0]))
        if len(second_sequence(s)[0]) < shortest:
            shortest = len(second_sequence(s)[0])
            candidates = [s]
        elif len(second_sequence(s)[0]) == shortest:
            candidates.append(s)
    print(candidates, shortest)
    print('---')


A 1
['A'] 1
---
>A 4
['>A'] 4
---
v<A 9
['v<A'] 9
---
vA 6
['vA'] 6
---
v>A 7
>vA 7
['v>A', '>vA'] 7
---
<A 8
['<A'] 8
---
A 1
['A'] 1
---
v<<A 10
<v<A 12
['v<<A'] 10
---
v<A 9
<vA 9
['v<A', '<vA'] 9
---
vA 6
['vA'] 6
---
>^A 7
['>^A'] 7
---
>^>A 10
>>^A 8
['>>^A'] 8
---
A 1
['A'] 1
---
>A 4
['>A'] 4
---
>>A 5
['>>A'] 5
---
^A 4
['^A'] 4
---
^>A 7
>^A 7
['^>A', '>^A'] 7
---
<A 8
['<A'] 8
---
A 1
['A'] 1
---
>A 4
['>A'] 4
---
^<A 9
<^A 9
['^<A', '<^A'] 9
---
^A 4
['^A'] 4
---
<<A 9
['<<A'] 9
---
<A 8
['<A'] 8
---
A 1
['A'] 1
---


In [90]:
unique_dir = dict()

for (start, end), seq in filtered_dir.items():
    if len(seq) == 1:
        unique_dir[(start, end)] = seq[0]

In [91]:
unique_dir # <<< looks like this still overcounts!

{((0, 1), (0, 1)): 'A',
 ((0, 1), (0, 2)): '>A',
 ((0, 1), (1, 0)): 'v<A',
 ((0, 1), (1, 1)): 'vA',
 ((0, 1), (1, 2)): 'v>A',
 ((0, 2), (0, 1)): '<A',
 ((0, 2), (0, 2)): 'A',
 ((0, 2), (1, 0)): 'v<<A',
 ((0, 2), (1, 1)): '<vA',
 ((0, 2), (1, 2)): 'vA',
 ((1, 0), (0, 1)): '>^A',
 ((1, 0), (0, 2)): '>>^A',
 ((1, 0), (1, 0)): 'A',
 ((1, 0), (1, 1)): '>A',
 ((1, 0), (1, 2)): '>>A',
 ((1, 1), (0, 1)): '^A',
 ((1, 1), (0, 2)): '>^A',
 ((1, 1), (1, 0)): '<A',
 ((1, 1), (1, 1)): 'A',
 ((1, 1), (1, 2)): '>A',
 ((1, 2), (0, 1)): '<^A',
 ((1, 2), (0, 2)): '^A',
 ((1, 2), (1, 0)): '<<A',
 ((1, 2), (1, 1)): '<A',
 ((1, 2), (1, 2)): 'A'}

In [96]:
char_positions = {'A':(0,2),
                  '^':(0,1),
                  'X':(0,0),
                  'v':(1,1),
                  '<':(1,0),
                  '>':(1,2)}



def sequence(code, start, end):
    result = ''

    if start == (0,0) or end == (0,0):
        #print("Invalid start or end")
        return '' 

    for char in code: # memoize here?
        end = char_positions[char]
        result += unique_dir[(start, end)]
        start = end
        
    return result


big_dir = dict()

# Precompute optimal sequences for 10 steps
for (start, end), seq in unique_dir.items():
    big_dir[(start, end)] = seq

    # Memoize? 
    for _ in range(1):
        seq = big_dir[(start, end)] 
        big_dir[(start, end)] = sequence(seq, start, end)

        


In [97]:
sequences = first_sequence(codes[0])

In [98]:
def get_bigdir_sequence(code, init_position = dir_position):
    memo = big_dir
    # Initialize the current position to the starting position
    start = init_position
   
    # Known paths: 
    result = ''

    # Go character by character in the code
    for char in code: 
        # Find the position of the character on the keypad
        end = char_positions[char]
        path_stub = memo[(start, end)]
        start = end

        result = result + path_stub

    return result

In [99]:
# Sanity check

result = 0 

for code in codes: 
    second_sequences = []

    for sequence in first_sequence(code):
        #print(sequence)
        second_sequences.append(get_bigdir_sequence(sequence))

    # compute the complexity 
    complexity = len(second_sequences[0]) * int(code[:-1])

    result += complexity

result # 157892 (overcount :/) 

180220